In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import cv2
from pandas.plotting import table
import matplotlib.pyplot as plt
from DataLoader import DataGenerator

/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorf

In [2]:
cte_height = 1080;
cte_width = 1920;

In [3]:
Informacoes = pd.read_csv("Depth_2").drop(columns=["Unnamed: 0"])
CU_size = 16
batch = 128
Informacoes.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97920000 entries, 0 to 97919999
Data columns (total 6 columns):
Origem    object
Frame     int64
Height    int64
Width     int64
QP        int64
Split     int64
dtypes: int64(5), object(1)
memory usage: 9.5 GB


In [4]:
Informacoes['Frame'] = Informacoes['Frame'].astype('uint16')
Informacoes['Height'] = Informacoes['Height'].astype('uint16')
Informacoes['Width'] = Informacoes['Width'].astype('uint16')
Informacoes['QP'] = Informacoes['QP'].astype('uint8')
Informacoes['Split'] = Informacoes['Split'].astype('uint8')

In [5]:
Informacoes["Height_Final"] = Informacoes["Height"]+CU_size
Informacoes["Width_Final"] = Informacoes["Width"]+CU_size
Informacoes.drop(Informacoes.loc[(Informacoes["Height_Final"]>cte_height)|(Informacoes["Width_Final"]>cte_width)].index,inplace=True)
Informacoes.drop(columns=['Height_Final',"Width_Final"],inplace=True)
Informacoes.loc[Informacoes['Origem'].str.contains("OldTown",na=False),"Origem"] = 1
print("OldTown")
Informacoes.loc[Informacoes['Origem'].str.contains("Crowd",na=False),"Origem"] = 2
print("Crowd")
Informacoes.loc[Informacoes['Origem'].str.contains("SunFlower",na=False),"Origem"] = 3
print("SunFlower")
Informacoes.loc[Informacoes['Origem'].str.contains("ParkJoy",na=False),"Origem"] = 4
print("ParkJoy")
Informacoes.loc[Informacoes['Origem'].str.contains("Ducks",na=False),"Origem"] = 5
print("Ducks")
Informacoes.loc[Informacoes['Origem'].str.contains("RushHour",na=False),"Origem"] = 6
print("RushHour")
Informacoes['Origem'] = Informacoes['Origem'].astype('uint8')
Informacoes.info(memory_usage='deep')

OldTown
Crowd
SunFlower
ParkJoy
Ducks
RushHour
<class 'pandas.core.frame.DataFrame'>
Int64Index: 96480000 entries, 0 to 97919997
Data columns (total 6 columns):
Origem    uint8
Frame     uint16
Height    uint16
Width     uint16
QP        uint8
Split     uint8
dtypes: uint16(3), uint8(3)
memory usage: 1.5 GB


In [6]:
parkjoy = np.fromfile('./Videos/ParkJoy_1920x1080_50_8bit_420.yuv',dtype='uint8')
rushhour= np.fromfile('./Videos/RushHour_1920x1080_25_8bit_420.yuv',dtype='uint8')
oldtown = np.fromfile('./Videos/OldTownCross_1920x1080_50_8bit_420.yuv',dtype='uint8')
ducks = np.fromfile('./Videos/DucksTakeOff_1920x1080_50_8bit_420.yuv',dtype='uint8')
crowdrun = np.fromfile('./Videos/Crowdrun_1920x1080_50_8bit_420.yuv',dtype='uint8')
sunflower = np.fromfile('./Videos/Sunflower_1920x1080_25_8bit_420.yuv',dtype='uint8')

In [7]:
Map = {1:oldtown,2:crowdrun,3:sunflower,4:parkjoy,5:ducks,6:rushhour}

In [8]:
Informacoes.head()

,Origem,Frame,Height,Width,QP,Split
0,3,427,0,0,22,0
1,3,427,0,16,22,0
2,3,427,16,0,22,0
3,3,427,16,16,22,0
4,3,427,0,32,22,0


In [9]:
def get_compiled_model(CU_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_dim=(CU_size*CU_size+1)),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model

model = get_compiled_model(CU_size)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
Informacoes = Informacoes.sample(frac=1)
training_set = Informacoes[:int(len(Informacoes)*0.8)]
validation_set = Informacoes[int(len(Informacoes)*0.8):]

In [11]:
training_generator = DataGenerator(training_set,Map,shuffle=False,dim = CU_size,batch_size=batch)
validation_generator = DataGenerator(validation_set,Map,shuffle=False,dim = CU_size,batch_size=batch)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                7740      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 22        
Total params: 9,002
Trainable params: 9,002
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit_generator(generator=training_generator,
                    validation_data = validation_generator,
                    use_multiprocessing=True,
                    workers=3,epochs=5)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
603000/603000 [==============================] - 50442s 84ms/step - loss: 0.2356 - acc: 0.9108 - val_loss: 0.2314 - val_acc: 0.9107
Epoch 2/5
603000/603000 [==============================] - 50739s 84ms/step - loss: 0.2315 - acc: 0.9110 - val_loss: 0.2309 - val_acc: 0.9108
Epoch 3/5
603000/603000 [==============================] - 50968s 85ms/step - loss: 0.2306 - acc: 0.9111 - val_loss: 0.2305 - val_acc: 0.9108
Epoch 4/5
603000/603000 [==============================] - 51511s 85ms/step - loss: 0.2302 - acc: 0.9111 - val_loss: 0.2301 - val_acc: 0.9109
Epoch 5/5
603000/603000 [==============================] - 51172s 85ms/step - loss: 0.2299 - acc: 0.9111 - val_loss: 0.2302 - val_acc: 0.9110


In [14]:
model.save("Model_Depth2_adam_91.h5")

In [18]:
oi = model.history

In [22]:
oi.history

{'loss': [0.23555226001577145,
  0.2315265213650826,
  0.23060743098762843,
  0.2301667556585265,
  0.22987626419773644],
 'acc': [0.9107631, 0.9109518, 0.9110676, 0.91110885, 0.9111211],
 'val_loss': [0.2314469504560503,
  0.2308897029382376,
  0.2304752748544833,
  0.23014208512336856,
  0.23016040107736344],
 'val_acc': [0.91066897, 0.9108163, 0.9108076, 0.91092724, 0.9110201]}

In [11]:
modelo = keras.models.load_model("Model_Depth0.h5")

In [25]:
oi = np.empty((1,64*64+1))
modelo.predict(oi)

IndexError: list index out of range